In [31]:
import pandas as pd 
def userdata(user_id: str):

    if not isinstance(user_id, str):
        return {'Mensaje': 'El argumento user_id debe ser una cadena de texto.'}

    df_user = pd.read_parquet('../CleanData/users_items.parquet', columns=['user_id', 'item_id', 'items_count'])
    df_user = df_user[df_user['user_id'].isin([user_id])]
    if df_user.empty:
        del df_user
        return {'Mensaje': 'Usuario no encontrado. Por favor ingrese un usuario válido'}
    
    df_steam = pd.read_parquet('../CleanData/steam_games.parquet', columns= ['id', 'price'])

    df = df_user.merge(df_steam, how='left', left_on= 'item_id', right_on= 'id')
    del df_user
    del df_steam

    dinero = df['price'].sum()
    cantidad_items = df['items_count'].max()

    del df

    df_reviews = pd.read_parquet('../CleanData/reviews.parquet', columns= ['user_id', 'recommend'])
    df_reviews = df_reviews[df_reviews['user_id'].isin([user_id])]
    recom = df_reviews['recommend'].sum()

    del df_reviews
    
    resultado = {
        'Usuario': user_id,
        'Dinero gastado': str(dinero) + ' USD',
        '% de recomendación': str(recom / cantidad_items) + ' %',
        'Cantidad de Items': int(cantidad_items)
    }

    return resultado

In [32]:
userdata('jonasdbomb')

df user
          user_id  item_id  items_count
16627  jonasdbomb       70          461
16628  jonasdbomb     1200          461
16629  jonasdbomb     1230          461
16630  jonasdbomb     1280          461
16631  jonasdbomb     3830          461
df
      user_id  item_id  items_count      id  price
0  jonasdbomb       70          461    70.0   9.99
1  jonasdbomb     1200          461  1200.0   9.99
2  jonasdbomb     1230          461  1230.0   0.00
3  jonasdbomb     1280          461  1280.0   0.00
4  jonasdbomb     3830          461  3830.0   9.99
df reviews
      user_id  recommend
0  jonasdbomb          1
      user_id  items_count  price  recommend
0  jonasdbomb          461   9.99          1
1  jonasdbomb          461   9.99          1
2  jonasdbomb          461   0.00          1
3  jonasdbomb          461   0.00          1
4  jonasdbomb          461   9.99          1
      user_id  items_count   price  recommend
0  jonasdbomb       212521  5352.1        461


{'Usuario': 'jonasdbomb',
 'Dinero gastado': '5352.099999999999 USD',
 '% de recomendación': '0.0 %',
 'Cantidad de Items': 212521}